# Перевод ранее сгенерированных вопросов на русский язык

In [1]:
from source.chroma_manager import ChromaDBManager, CustomEmbeddingFunction
from source.llm_manager import LMStudioClient
from prompt.prompts import QuestionsToDoc
from tqdm import tqdm
import ast
import json
import logging

DB_ENG_CONFIG_PATH = "config/embedding/questions_gen.yaml"
DB_RUS_CONFIG_PATH = "config/embedding/questions_rus.yaml"
MODEL_CONFIG_PATH = "config/model_question_gen.yaml"


logging.basicConfig(filename="log/question_gen.log", level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s")


# Инициализация менеджеров
eng_chroma = ChromaDBManager(DB_ENG_CONFIG_PATH)
rus_chroma = ChromaDBManager(DB_RUS_CONFIG_PATH)

llm = LMStudioClient(MODEL_CONFIG_PATH)  # URL настроить под свой сервер

# Извлечение всех документов
all_documents = eng_chroma.collection.get()
doc_ids = all_documents.get("ids", [])
doc_texts = all_documents.get("documents", [])
doc_metadata = all_documents.get("metadatas", [])

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


e:\ImportantFiles\Documents\University\Magic App\source\llm_manager.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [2]:
def safe_str_to_dict(s: str):
    """
    Преобразует строку в словарь, если возможно.
    В противном случае возвращает False.
    """
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return result
        return False
    except (ValueError, SyntaxError):
        return False

In [3]:
# Список ids где в метадате отсутвуют вопросы
rus_collection = rus_chroma.collection.get()
rus_ids = rus_collection.get("ids", [])
ids_to_generate = [x for x in doc_ids if x not in rus_ids]
len(ids_to_generate)

426

In [4]:
ids_to_generate = []

In [ ]:
for doc_id, doc_text, metadata in tqdm(zip(doc_ids, doc_texts, doc_metadata)):
    if doc_id not in ids_to_generate:
        questions_eng = eval(metadata['questions'])
        
        question_1 = questions_eng['question_1']
        answer_1 = questions_eng['answer_1']

        # Генерация вопросов через LLM
        prompt = QuestionsToDoc.TRANSLATE_TO_RUSSIAN.format(
            question=question_1,
            answer=answer_1,
            doc=doc_text
        )
        # print(repr(prompt))  # Выведет строку в raw-формате
        questions = llm.post_completion(user_input=prompt)
        llm.clear_context() # очищаем контекст
        questions_dict = safe_str_to_dict(questions)
        if questions_dict == False:
            prompt = QuestionsToDoc.CORRECTING_DICKTIONARY_ONE_QUESTION.format(input=questions)
            questions = llm.post_completion(user_input=prompt)
            llm.clear_context() # очищаем контекст
            questions_dict = safe_str_to_dict(questions)
            if questions_dict == False:
                print("Словарь не сгенерирован")
                print(questions)
                logging.error(f"doc_id={doc_id}: questions_dict={questions}")
                continue
            
        # Обновление метаданных
        # print("Успешно")
        # print(questions_dict)
        metadata["questions"] = json.dumps(questions_dict)
        rus_chroma.insert_document(document_id=doc_id, document_text=doc_text, metadata=metadata)

1it [00:56, 56.06s/it]